In [ ]:
import os

# CUDA 디바이스 0, 2만 사용하도록 설정
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
from dataclasses import dataclass

import torch
from datasets import Dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    BitsAndBytesConfig,
    PreTrainedTokenizerBase
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("RowKick/deberta-v3-base-korean-stage2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'
tokenizer.truncation_side='right'
tokenizer.add_eos_token = True

In [ ]:

model = AutoModelForSequenceClassification.from_pretrained(
    "RowKick/deberta-v3-base-korean-stage2",
    num_labels=1,  
    device_map="cuda:0",
)


model.config.use_cache = False
model

In [ ]:
ds = Dataset.from_csv("test.csv")

In [ ]:
class CustomTokenizer:
    def __init__(
        self,
        tokenizer: PreTrainedTokenizerBase,
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, batch: dict) -> dict:
        title = ["<Title>: " + t for t in batch["title"]]
        para = ["\n\n<Full text>: " + t for t in batch["paragraph_text"]]
        texts = [t + p for t, p in zip(title, para)]
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)
        tokenized['prompt'] = texts
        return {**tokenized}

In [ ]:
encode = CustomTokenizer(tokenizer, max_length=512)
ds = ds.map(encode, batched=True)

In [ ]:
from tqdm.notebook import tqdm
from transformers.data.data_collator import pad_without_fast_tokenizer_warning


@torch.no_grad()
@torch.cuda.amp.autocast()
def inference(ds, model, batch_size=1):
    preds = []
    pseudo = []
    model.eval()
    
    for start_idx in tqdm(range(0, len(ds), batch_size)):
        end_idx = min(start_idx + batch_size, len(ds))
        tmp = ds[start_idx:end_idx]
        input_ids = tmp["input_ids"]
        attention_mask = tmp["attention_mask"]
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        outputs = model(**inputs.to("cuda:0"))
        # proba = outputs.logits.cpu()
        logits = outputs.logits.squeeze(-1).cpu()
        proba = logits

        
        # preds.extend(proba[:, 0].tolist())
        # pseudo.extend(proba[:,1].tolist())
        preds.extend(proba.tolist())  # logit 하나

    
    return preds, pseudo

In [ ]:
probs, _ = inference(ds, model)  

import pandas as pd
sub = pd.read_csv('sample_submission.csv')

sub['generated'] = probs  

sub.to_csv('stage2_regularization.csv', index=False)